In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
from flask import Flask,render_template,redirect
from flask_pymongo import PyMongo
import requests
import pandas as pd

# Step 1 - Scraping

### NASA Mars News

In [2]:
news_url = 'https://mars.nasa.gov/news/'
executable_path = {'executable_path': 'chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)
browser.visit(news_url)
browser.is_element_present_by_css("ul.item_list li.slide", wait_time=1)
news_html = browser.html
soup = bs(news_html, 'lxml')
result = soup.select_one("ul.item_list li.slide")
news_title = result.find("div", class_= "content_title").text
print(news_title)
news_paragraph = result.find("div", class_ = "article_teaser_body").text
print(news_paragraph)

NASA Engineers Checking InSight's Weather Sensors
An electronics issue is suspected to be preventing the sensors from sharing their data about Mars weather with the spacecraft.


In [3]:
# url = 'https://mars.nasa.gov/news/'
# response = requests.get(url)
# soup = bs(response.text, 'lxml')
# result = soup.select_one("ul.item_list li.slide")
# news_title = result.find("div", class_= "content_title").text
# print(title)
# news_paragraph = result.find("div", class_ = "article_teaser_body").text
# print(news_paragraph)

### JPL Mars Space Images - Featured Image

In [4]:
featue_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
executable_path = {'executable_path': 'chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)
browser.visit(featue_url)
featue_html = browser.html
soup = bs(featue_html, 'html.parser')
featured_group = soup.find("article", class_= "carousel_item")
part_image_url = featured_group.find('a', class_='button fancybox')['data-fancybox-href']
featured_image_url = f"https://www.jpl.nasa.gov{part_image_url}"
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA19952_ip.jpg


### Mars Facts

In [5]:
facts_url = 'https://space-facts.com/mars/'
tables = pd.read_html(facts_url)
facts_df = tables[0]
facts_df.columns = ['MARS PLANT PROFILE', 'Parameter']
facts_df

,MARS PLANT PROFILE,Parameter
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [11]:
facts_df.to_html("facts_table.html")
facts_html = facts_df.to_html(index=False)
facts_html = facts_html.replace('\n','')
facts_html

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th>MARS PLANT PROFILE</th>      <th>Parameter</th>    </tr>  </thead>  <tbody>    <tr>      <td>Equatorial Diameter:</td>      <td>6,792 km</td>    </tr>    <tr>      <td>Polar Diameter:</td>      <td>6,752 km</td>    </tr>    <tr>      <td>Mass:</td>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <td>Moons:</td>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <td>Orbit Distance:</td>      <td>227,943,824 km (1.38 AU)</td>    </tr>    <tr>      <td>Orbit Period:</td>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <td>Surface Temperature:</td>      <td>-87 to -5 °C</td>    </tr>    <tr>      <td>First Record:</td>      <td>2nd millennium BC</td>    </tr>    <tr>      <td>Recorded By:</td>      <td>Egyptian astronomers</td>    </tr>  </tbody></table>'

### Mars Hemispheres

In [13]:
hemi_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
executable_path = {'executable_path': 'chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)
browser.visit(hemi_url)
hemi_html = browser.html
soup = bs(hemi_html, 'lxml')
results = browser.find_by_css('div.description a.product-item')
hemisphere_image_urls = []
for i in range(len(results)):
    browser.find_by_css('div.description a.product-item')[i].click()
    img = browser.find_by_text("Sample").first
    img_url = img['href']
    img_title = browser.find_by_css('h2.title').text
    info = {"title":img_title, "img_url":img_url}
    hemisphere_image_urls.append(info)
    browser.back()
    
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]

In [16]:
hemisphere_image_urls[1]['title']

'Schiaparelli Hemisphere Enhanced'

# Step 2 - MongoDB and Flask Application